In [1]:
# Generate the video for the env with fake action

In [3]:
import os
import torch
import gymnasium as gym

In [4]:
import torchdriveenv
from torchdriveenv.env_utils import load_default_env_config, load_default_validation_data

/home/kezhang/work/torchdriveenv/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Commercial access denied and fallback to check for academic access.


In [5]:
data = load_default_validation_data()
env_config = load_default_env_config()

In [6]:
env_config.render_mode="video"

In [7]:
env = gym.make('torchdriveenv-v0', args={'cfg': env_config, 'data': data})

INFO:torchdriveenv.gym_env:seed: 1075771777
INFO:torchdriveenv.gym_env:    def get_reward(self):
        x = self.simulator.get_state()[..., 0]
        y = self.simulator.get_state()[..., 1]
        psi = self.simulator.get_state()[..., 2]

        d = math.dist((x, y), (self.last_x, self.last_y)) if (self.last_x is not None) and (self.last_y is not None) else 0
        distance_reward = 1 if d > 0.5 else 0
        psi_reward = (1 - math.cos(psi - self.last_psi)) * (-20.0) if (self.last_psi is not None) else 0
        if self.check_reach_target():
            reach_target_reward = 10
            self.reached_waypoint_num += 1
        else:
            reach_target_reward = 0
        r = torch.zeros_like(x)
        r += reach_target_reward + distance_reward + psi_reward
        return r.item()



In [8]:
def main(cfg, env):
    env.reset()
    while True:
        actions = torch.tensor([1, 0], dtype=torch.float32, device='cpu')  # accelerate hard without steering
        obs, reward, terminated, truncated, info = env.step(actions)  
        if terminated or truncated:
            break
    env.close()

In [9]:
main(env_config, env)

### 